In [8]:
# Cargamos la librería de pandas.
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [10]:
# Cargamos el dataset original.
df = pd.read_csv('Restaurant reviews.csv')

# Seleccionamos las columnas que utilizaremos en el modelo.
nuevas_columnas = ['Review', 'Rating', 'Metadata']

# Eliminamos el resto de columnas del dataset.
reviews = df[nuevas_columnas]

# Extraemos el número de Reviews y Followers de la columna Metadata en dos nuevas columnas.
# El razonamiento detrás de esta transformación se encuentra en el notebook '1.0.1 Análisis Metadata'.
reviews[['Reviews', 'Followers']] = reviews['Metadata'].str.extract('(\d+)\D*(\d*)', expand=True)
reviews[['Reviews', 'Followers']] = reviews[['Reviews', 'Followers']].apply(pd.to_numeric)
reviews['Reviews'].fillna(0, inplace=True)
reviews['Followers'].fillna(0, inplace=True)
reviews = reviews.drop('Metadata', axis=1)

In [ ]:
# Generamos una valoración positiva, negativa o neutral para cada reseña. Debido al excesivo tiempo de computación
# que requiere un LLM, se ha optado por emplear un modelo alternativo disponible en la librería Transformers de Hugging Face.

from transformers import pipeline
modelo_sentimientos = pipeline(model = "nlptown/bert-base-multilingual-uncased-sentiment")

# Generamos la columna Sentimiento sobre la que guardaremos la valoración positiva, negativa o neutral de cada reseña.
reviews['Sentimiento'] = ''
# El modelo seleccionado nos devolverá un sentimiento entre 1 y 5. Consideraremos negativas las valoraciones 1 y 2, positivas
# las valoraciones 4 y 5, y neutrales las valoradas con 3. Para que los datos se adecúen al formato input de un árbol de
# decisión, la variable será numérica considerando -1 como negativo, 0 como neutral y 1 como positivo.
mapping = {
    '1 star': '-1',
    '2 stars': '-1',
    '3 stars': '0',
    '4 stars': '1',
    '5 stars': '1'
}

# Iteramos sobre cada reseña para obtener su sentimiento.
for i in range(0, len(reviews)):
    # Si la reseña es demasiado larga, el modelo no será capaz de analizarla y mostrará un Error. En tales casos, no
    # se añadirá un sentimiento a la reseña.
    try:
        # Seleccionamos una reseña.
        reseña = reviews['Review'].iloc[i]
        # Aplicamos el modelo, que nos devolverá un diccionario con una valoración del sentimiento.
        dict_valoracion = modelo_sentimientos(reseña)
        # Extraemos la valoración del sentimiento (1 a 5) fuera del diccionario.
        valoracion = dict_valoracion[0]['label']
        # Transformamos la valoración a sentimiento positivo/negativo/neutral.
        sentimiento = mapping[valoracion]
        # Guardamos el sentimiento en la fila correspondiente del dataframe.
        reviews['Sentimiento'].iloc[i] = sentimiento
    # En caso de obtener Error por el tamaño de la reseña. No añadimos ningún valor en la fila correspondiente del dataframe.
    except:
        reviews['Sentimiento'].iloc[i] = float('nan')
    
# Desestimaremos aquellas reseñas para las que no haya sido posible obtener un sentimiento. Dado el tamaño del dataset,
# consideramos que esta opción resulta más adecuada que otorgar una valoración neutral a tales reseñas.
reviews.dropna(subset=['Sentimiento'], inplace=True)
    
# Eliminamos la columna Review.
reviews = reviews.drop('Review', axis=1)

Some layers from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment were not used when initializing TFBertForSequenceClassification: ['dropout_37']
- This IS expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertForSequenceClassification were initialized from the model checkpoint at nlptown/bert-base-multilingual-uncased-sentiment.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertForSequenceClassification for predictions without further training.


In [4]:
# Observamos si hay alguna reseña con un Rating no numérico.
ratings_no_numericos = reviews['Rating'].apply(pd.to_numeric, errors='coerce').isna()
print(reviews[ratings_no_numericos])
# Observamos como aparece una reseña sin valoración numérica. Eliminamos tal fila.
reviews['Rating']=reviews['Rating'].apply(pd.to_numeric, errors='coerce')
reviews.dropna(subset=['Rating'], inplace=True)

     Rating  Reviews  Followers Sentimiento
7601   Like     12.0       21.0           1


In [5]:
# Guardamos las transformaciones en el dataset reviews en un csv.
reviews.to_csv('datos_procesados.csv', sep=';', index=False)